In [1]:
import gymnasium as gym
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
gym.make("Blackjack-v1", natural = True , sab = False )

<OrderEnforcing<PassiveEnvChecker<BlackjackEnv<Blackjack-v1>>>>